In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
                transforms.RandomVerticalFlip(),  
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05)
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 128
p1 = .25


class ResBlock(nn.Module):
    def __init__(self, in_size, kernel_size, stride, padding, padding_mode, encoder=False):
        super().__init__()
        if (encoder):
            self.conv1 = nn.Conv2d(in_size, in_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
        else:
            self.conv1 = nn.ConvTranspose2d(in_size, in_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(in_size)

    def convblock(self, x):
        x = F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        return x
   
    def forward(self, x): 
        return x + self.convblock(x)
    
    
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.resblock1 = ResBlock(in_size=1, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode, encoder=True)
        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.resblock2 = ResBlock(in_size=channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode, encoder=True)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.resblock3 = ResBlock(in_size=channels_base * 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.resblock4 = ResBlock(in_size=channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.resblock5 = ResBlock(in_size=channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm5 = nn.BatchNorm2d(2)
        
    def forward(self, input):
        # encoder
        x = self.resblock1(input)
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = y = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.resblock2(x)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = self.resblock3(x)
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = self.resblock4(x + y)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = self.resblock5(x + input)
        x = F.leaky_relu(self.batchnorm5(self.convtrans3(x)), negative_slope=0.1)


        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 1, 32, 32]              10
       BatchNorm2d-2            [-1, 1, 32, 32]               2
            Conv2d-3            [-1, 1, 32, 32]              10
       BatchNorm2d-4            [-1, 1, 32, 32]               2
            Conv2d-5            [-1, 1, 32, 32]              10
       BatchNorm2d-6            [-1, 1, 32, 32]               2
          ResBlock-7            [-1, 1, 32, 32]               0
            Conv2d-8          [-1, 128, 32, 32]           1,280
       BatchNorm2d-9          [-1, 128, 32, 32]             256
           Conv2d-10          [-1, 128, 16, 16]         147,584
      BatchNorm2d-11          [-1, 128, 16, 16]             256
           Conv2d-12          [-1, 128, 16, 16]         147,584
      BatchNorm2d-13          [-1, 128, 16, 16]             256
           Conv2d-14          [-1, 128,

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00293561 (0.01709816), PSNR 25.32302284 (20.52414465), SSIM 0.74731821 (0.64117797)
Finished training epoch 0
Validate: MSE 0.00322036 (0.00278045), PSNR 24.92094803 (25.63443155), SSIM 0.70469999 (0.77509621)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00326500 (0.00272957), PSNR 24.86116409 (25.66183827), SSIM 0.73181009 (0.76726406)
Finished training epoch 1
Validate: MSE 0.00318554 (0.00275969), PSNR 24.96817207 (25.66342915), SSIM 0.69989467 (0.76932061)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00229017 (0.00265431), PSNR 26.40131378 (25.78565632), SSIM 0.79573667 (0.76688160)
Finished training epoch 2
Validate: MSE 0.00310053 (0.00277151), PSNR 25.08563805 (25.65073160), SSIM 0.69363195 (0.76751489)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00258860 (0.00256661), PSNR 25.86934662 (25.92577371), SSIM 0.77326733 (0.76655343)
Finished training epoch 3
Validate: MSE 0.00301182 (0.0

Epoch: 30, MSE 0.00249505 (0.00225539), PSNR 26.02920914 (26.49089394), SSIM 0.77060914 (0.78268024)
Finished training epoch 30
Validate: MSE 0.00338900 (0.00283043), PSNR 24.69927788 (25.55449123), SSIM 0.69099444 (0.76346488)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00237945 (0.00224836), PSNR 26.23523903 (26.50513915), SSIM 0.76919937 (0.78324384)
Finished training epoch 31
Validate: MSE 0.00306742 (0.00271462), PSNR 25.13227272 (25.73147482), SSIM 0.69728243 (0.76221051)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00333059 (0.00224785), PSNR 24.77478409 (26.50585469), SSIM 0.76739538 (0.78340658)
Finished training epoch 32
Validate: MSE 0.00366157 (0.00295347), PSNR 24.36332130 (25.38209616), SSIM 0.69073045 (0.76111890)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00224181 (0.00224148), PSNR 26.49401474 (26.51791386), SSIM 0.78397483 (0.78344670)
Finished training epoch 33
Validate: MSE 0.00357651 (0.00282201), PSNR 

Epoch: 60, MSE 0.00251157 (0.00212717), PSNR 26.00055122 (26.74484094), SSIM 0.78342092 (0.78748436)
Finished training epoch 60
Validate: MSE 0.00401021 (0.00329890), PSNR 23.96832466 (24.91196065), SSIM 0.68547338 (0.75900547)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00229018 (0.00212815), PSNR 26.40129662 (26.74453485), SSIM 0.80767649 (0.78749774)
Finished training epoch 61
Validate: MSE 0.00318300 (0.00276295), PSNR 24.97163200 (25.65586133), SSIM 0.69010913 (0.75950010)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00196327 (0.00212026), PSNR 27.07019234 (26.76050181), SSIM 0.78353482 (0.78777250)
Finished training epoch 62
Validate: MSE 0.00358700 (0.00308035), PSNR 24.45268631 (25.20108433), SSIM 0.68891454 (0.76052809)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00193414 (0.00212605), PSNR 27.13511658 (26.74636424), SSIM 0.78746742 (0.78784874)
Finished training epoch 63
Validate: MSE 0.00341783 (0.00293962), PSNR 

Epoch: 90, MSE 0.00176739 (0.00205368), PSNR 27.52666283 (26.89665324), SSIM 0.81213742 (0.78946289)
Finished training epoch 90
Validate: MSE 0.00379104 (0.00316767), PSNR 24.21241379 (25.08878080), SSIM 0.69987655 (0.76879937)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00181527 (0.00205540), PSNR 27.41057968 (26.89509001), SSIM 0.80213624 (0.78939435)
Finished training epoch 91
Validate: MSE 0.00370430 (0.00345964), PSNR 24.31293297 (24.70420958), SSIM 0.68485212 (0.74936244)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00202736 (0.00206152), PSNR 26.93068886 (26.88269410), SSIM 0.79752034 (0.78948793)
Finished training epoch 92
Validate: MSE 0.00431656 (0.00361442), PSNR 23.64862251 (24.53792420), SSIM 0.68200141 (0.75434613)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00184870 (0.00204726), PSNR 27.33134079 (26.91060820), SSIM 0.78566891 (0.78970102)
Finished training epoch 93
Validate: MSE 0.00398022 (0.00322401), PSNR 

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00371300 (0.00309779), PSNR 24.30274963 (25.17699399), SSIM 0.68639827 (0.76088898)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()